# 📊 Netflix 데이터 시각화 - 전처리 & 시각화

**목표**: Netflix 데이터를 전처리하고 matplotlib으로 시각화하기

**시간**: 60분

---

## Section 1: 데이터 로드 (10분)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
df = pd.read_csv('../data/netflix_titles.csv')

# 기본 정보 확인
print(f"데이터 크기: {len(df):,}행 x {len(df.columns)}열")
print(f"\n{'='*50}")
df.head()

In [ ]:
# 데이터 정보 확인
df.info()

---

## Section 2: 데이터 탐색 (10분)

In [ ]:
# 결측치 현황
print("=== 결측치 현황 ===")
missing = df.isnull().sum()
print(missing[missing > 0])  # 결측치가 있는 컬럼만 표시

print(f"\n결측치 비율 (%)")
missing_pct = (df.isnull().sum() / len(df)) * 100
print(missing_pct[missing_pct > 0].sort_values(ascending=False))

In [ ]:
# 기본 통계
print("\n=== 기본 통계 ===")
df.describe()

In [ ]:
# 콘텐츠 유형 확인
print("\n=== 콘텐츠 정보 ===")
print(f"콘텐츠 유형: {df['type'].unique()}")
print(f"제작 국가 수: {df['country'].nunique()}")
print(f"시청 등급 종류: {df['rating'].nunique()}")
print(f"\n시청 등급별 분포:")
print(df['rating'].value_counts().head(10))

---

## Section 3: 결측치 처리 (15분)

In [ ]:
print("📋 결측치 처리 전:")
print(f"총 행 수: {len(df):,}\n")

# director 결측치 처리
df['director'] = df['director'].fillna('Unknown')
print(f"✅ director: Unknown으로 채움 ({df['director'].isnull().sum()} 남음)")

# cast 결측치 처리
df['cast'] = df['cast'].fillna('Unknown')
print(f"✅ cast: Unknown으로 채움 ({df['cast'].isnull().sum()} 남음)")

# country 결측치 처리
df['country'] = df['country'].fillna('Unknown')
print(f"✅ country: Unknown으로 채움 ({df['country'].isnull().sum()} 남음)")

# date_added 결측치 제거
print(f"\n⚠️ date_added: {df['date_added'].isnull().sum()}개 행 제거")
df = df.dropna(subset=['date_added'])
print(f"✅ 제거 후 총 행 수: {len(df):,}행")

# rating 결측치 처리
df['rating'] = df['rating'].fillna('Not Rated')
print(f"✅ rating: Not Rated로 채움 ({df['rating'].isnull().sum()} 남음)")

print(f"\n{'='*50}")
print(f"✅ 결측치 처리 완료: {len(df):,}행 남음")

---

## Section 4: 파생변수 생성 (15분)

### 파생변수 6개 생성하기

### 4.1 연대 (decade)

In [ ]:
df['decade'] = (df['release_year'] // 10) * 10
print(f"✅ decade 컬럼 추가")
print(f"연대별 분포: {sorted(df['decade'].unique())}")
print(f"\n샘플:")
print(df[['title', 'release_year', 'decade']].head(5))

### 4.2 한국 콘텐츠 여부 (is_korean)

In [ ]:
df['is_korean'] = df['country'].str.contains('South Korea', na=False)
korean_count = df['is_korean'].sum()
print(f"✅ is_korean 컬럼 추가: 한국 콘텐츠 {korean_count:,}개")
print(f"\n샘플 (한국 콘텐츠):")
print(df[df['is_korean']][['title', 'country']].head(5))

### 4.3 제목 길이 (title_length)

In [ ]:
df['title_length'] = df['title'].str.len()
print(f"✅ title_length 컬럼 추가")
print(f"평균 제목 길이: {df['title_length'].mean():.1f}자")
print(f"최대/최소 제목 길이: {df['title_length'].max()}/{df['title_length'].min()}자")
print(f"\n샘플:")
print(df[['title', 'title_length']].head(5))

### 4.4 콘텐츠 나이 (content_age)

In [ ]:
df['content_age'] = 2024 - df['release_year']
print(f"✅ content_age 컬럼 추가")
print(f"평균 콘텐츠 나이: {df['content_age'].mean():.1f}년")
print(f"최대/최소 콘텐츠 나이: {df['content_age'].max()}/{df['content_age'].min()}년")
print(f"\n샘플:")
print(df[['title', 'release_year', 'content_age']].head(5))

### 4.5 최근 콘텐츠 여부 (is_recent)

In [ ]:
df['is_recent'] = df['release_year'] >= 2019
recent_count = df['is_recent'].sum()
print(f"✅ is_recent 컬럼 추가")
print(f"최근 5년(2019~2024) 콘텐츠: {recent_count:,}개 ({recent_count/len(df)*100:.1f}%)")
print(f"\n샘플 (최근 콘텐츠):")
print(df[df['is_recent']][['title', 'release_year']].head(5))

### 4.6 출연진 정보 유무 (has_cast)

In [ ]:
df['has_cast'] = df['cast'] != 'Unknown'
has_cast_count = df['has_cast'].sum()
print(f"✅ has_cast 컬럼 추가")
print(f"출연진 정보 있음: {has_cast_count:,}개 ({has_cast_count/len(df)*100:.1f}%)")
print(f"출연진 정보 없음: {(~df['has_cast']).sum():,}개")

---

## Section 5: matplotlib 시각화 (20분)

### 5.0 라이브러리 설정

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 시각화 라이브러리 로드 완료")

### 5.1 제목 길이 히스토그램

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['title_length'], bins=30, color='#E50914', alpha=0.7, edgecolor='black')
plt.xlabel('제목 길이 (글자 수)', fontsize=12)
plt.ylabel('개수', fontsize=12)
plt.title('📏 Netflix 콘텐츠 제목 길이 분포', fontsize=14, fontweight='bold')

avg_length = df['title_length'].mean()
plt.axvline(avg_length, color='black', linestyle='--', linewidth=2,
            label=f'평균: {avg_length:.1f}자')
plt.legend(fontsize=11)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"💡 평균 제목 길이: {avg_length:.1f}자")
print(f"💡 최대/최소: {df['title_length'].max()}/{df['title_length'].min()}자")

### 5.2 연대별 막대그래프

In [ ]:
decade_counts = df['decade'].value_counts().sort_index().tail(10)

plt.figure(figsize=(12, 6))
bars = plt.bar(decade_counts.index, decade_counts.values, 
               color='#E50914', alpha=0.8, edgecolor='black')

# 최대값인 막대를 다른 색으로
max_idx = decade_counts.values.argmax()
bars[max_idx].set_color('#8B0000')

plt.xlabel('연대', fontsize=12)
plt.ylabel('콘텐츠 수', fontsize=12)
plt.title('📅 연대별 Netflix 콘텐츠 제작량', fontsize=14, fontweight='bold')
plt.xticks(decade_counts.index, rotation=45)
plt.grid(axis='y', alpha=0.3)

# 각 막대 위에 값 표시
for decade, count in zip(decade_counts.index, decade_counts.values):
    plt.text(decade, count + 50, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

peak_decade = decade_counts.idxmax()
print(f"💡 {peak_decade}년대에 가장 많은 {decade_counts.max():,}개 제작")

### 5.3 한국 콘텐츠 파이차트

In [ ]:
korean_counts = df['is_korean'].value_counts()

plt.figure(figsize=(8, 8))
colors = ['#564d4d', '#E50914']
explode = (0, 0.1)

plt.pie(korean_counts.values, 
        labels=['비한국 콘텐츠', '한국 콘텐츠'],
        autopct='%1.1f%%',
        colors=colors,
        explode=explode,
        startangle=90,
        textprops={'fontsize': 12, 'fontweight': 'bold'})

plt.title('🇰🇷 한국 콘텐츠 비율', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

korean_pct = (df['is_korean'].sum() / len(df)) * 100
print(f"💡 한국 콘텐츠 비율: {korean_pct:.1f}%")

### 5.4 Movie vs TV Show 막대그래프

In [ ]:
type_counts = df['type'].value_counts()

plt.figure(figsize=(10, 6))
bars = plt.bar(type_counts.index, type_counts.values, 
               color=['#E50914', '#564d4d'], alpha=0.8, 
               edgecolor='black', width=0.6)

plt.xlabel('콘텐츠 유형', fontsize=12)
plt.ylabel('개수', fontsize=12)
plt.title('🎬 Movie vs TV Show 분포', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

for bar, count in zip(bars, type_counts.values):
    height = bar.get_height()
    pct = (count / len(df)) * 100
    plt.text(bar.get_x() + bar.get_width()/2., height + 100,
             f'{count:,}개\n({pct:.1f}%)',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

movie_pct = (type_counts.get('Movie', 0) / len(df)) * 100
tv_pct = (type_counts.get('TV Show', 0) / len(df)) * 100
print(f"💡 영화: {type_counts.get('Movie', 0):,}개 ({movie_pct:.1f}%)")
print(f"💡 TV쇼: {type_counts.get('TV Show', 0):,}개 ({tv_pct:.1f}%)")

---

## Section 6: 데이터 저장

In [ ]:
df.to_csv('../data/netflix_cleaned.csv', index=False)

print('✅ 데이터 저장 완료: data/netflix_cleaned.csv')
print(f'\n최종 데이터: {len(df):,}행 x {len(df.columns)}열')
print(f'\n파생변수 6개:')
print('  1. decade - 연대')
print('  2. is_korean - 한국 콘텐츠 여부')
print('  3. title_length - 제목 길이')
print('  4. content_age - 콘텐츠 나이')
print('  5. is_recent - 최근 5년 여부')
print('  6. has_cast - 출연진 정보 유무')

---

## Section 7: 다음 단계 안내

## 🎯 다음 단계: Streamlit 대시보드로!

### ✅ 지금까지 한 일
- ✅ 데이터 로드 및 탐색
- ✅ 결측치 처리
- ✅ 파생변수 생성 (6개)
- ✅ matplotlib으로 시각화 (4개 그래프)

### 📊 이제 할 일
**app.py에서 이 코드를 Streamlit + Plotly로 변환하기!**

#### matplotlib → Plotly 변환 예시:

**Before (Jupyter)**
```python
plt.hist(df['title_length'], bins=30)
```

**After (Streamlit)**
```python
import plotly.express as px
fig = px.histogram(df, x='title_length', nbins=30)
st.plotly_chart(fig)
```

---

**준비됐나요? 이제 `app.py`로 이동해서 인터랙티브 대시보드를 만들어봅시다!** 🚀